# Calculate MS properties

## 0. Necessary Packages

In [ ]:
import logging
import shutil
import os
import yaml
from typing import Optional

import numpy as np
import matplotlib.pyplot as plt

from arkane.input import process_model_chemistry
from arkane.common import symbol_by_number, get_principal_moments_of_inertia
from arkane.encorr.corr import assign_frequency_scale_factor, get_atom_correction
from arkane.ess import ess_factory
from arkane.modelchem import LOT, LevelOfTheory, CompositeLevelOfTheory, model_chem_to_lot

from rmgpy import constants
from rmgpy.kinetics.tunneling import Eckart
from rmgpy.molecule.element import get_element
from rmgpy.molecule.molecule import Molecule
from rmgpy.qm.qmdata import QMData
from rmgpy.qm.symmetry import PointGroupCalculator
from rmgpy.species import Species, TransitionState
from rmgpy.statmech import (IdealGasTranslation,
                            NonlinearRotor,
                            LinearRotor,
                            HarmonicOscillator,
                            Conformer)

from acs.common import read_yaml_file

%matplotlib inline

In [ ]:
def get_symmetry(coords, atom_numbers, scr_dir=None):

    scr_dir = scr_dir or os.path.join('.', 'scratch')
    os.makedirs(scr_dir, exist_ok=True)
    
    symmetry = optical_isomers = 1
    try:
        qmdata = QMData(
            groundStateDegeneracy=1,  # Only needed to check if valid QMData
            numberOfAtoms=len(atom_numbers),
            atomicNumbers=atom_numbers,
            atomCoords=(coords, 'angstrom'),
            energy=(0.0, 'kcal/mol')  # Only needed to avoid error
        )
        settings = type('', (),
                        dict(symmetryPath='symmetry',
                             scratchDirectory=scr_dir))()
        pgc = PointGroupCalculator(settings, '0', qmdata)  # '0' is an unique id used for calculator
        pg = pgc.calculate()
        if pg is not None:
            symmetry = pg.symmetry_number
            optical_isomers = 2 if pg.chiral else 1
            logging.debug(f"Symmetry algorithm found {optical_isomers} optical isomers "
                          f"and a symmetry number of {symmetry}")
        else:
            logging.warning("Symmetry algorithm errored when computing point group. "
                            "Using symmetry number=1 and optical isomers = 1 for "
                            "further calculations, which may not be true.")
        return symmetry, optical_isomers
    finally:
        shutil.rmtree(scr_dir)

def get_lot_and_freq_scale(energy_level: str,
                           freq_level: str,
                           energy_software: str = 'gaussian',
                           freq_scale: Optional[float] = None):
    # Get energy level and assign software
    energy_level = process_model_chemistry(energy_level)
    energy_software = energy_software or energy_log.get_software()
    energy_level = energy_level.update(software=energy_software)

    # Get freq level
    freq_level = process_model_chemistry(freq_level)

    # Assign level of theory and frequency scale factor
    if energy_level.to_model_chem() in ['cbsqb3'] \
           or energy_level == freq_level:
        level_of_theory = energy_level
    else:
        level_of_theory = CompositeLevelOfTheory(freq=freq_level, energy=energy_level)
    freq_scale = freq_scale or assign_frequency_scale_factor(level_of_theory)
    return level_of_theory, freq_scale

def get_rotational_mode(coords, number, external_symmetry=None):
    if not external_symmetry:
        external_symmetry, _ = get_symmetry(coords, number)
    
    # Rotational
    moments_of_inertia = get_principal_moments_of_inertia(coords=coords,
                                                          numbers=number,)[0]
    if any([moment_of_inertia == 0.0 for moment_of_inertia in moments_of_inertia]):
        # this is a linear rotor
        moments_of_inertia = [moment_of_inertia for moment_of_inertia in moments_of_inertia
                              if moment_of_inertia != 0.0]
        if abs(moments_of_inertia[0] - moments_of_inertia[1]) > 0.01:
            raise Exceptions(f'Expected two identical moments of inertia for a linear rigis rotor, '
                             f'but got {moments_of_inertia}')
        return LinearRotor(inertia=(moments_of_inertia[0], "amu*angstrom^2"),
                           symmetry=external_symmetry)
    else:
        # this is a non-linear rotor
        return NonlinearRotor(inertia=(moments_of_inertia, "amu*angstrom^2"),
                              symmetry=external_symmetry)
    
def get_element_counts(number):
    # Get atoms count
    atoms = {}
    for atom_num in number:
        try:
            symbol = symbol_by_number[atom_num]
        except KeyError:
            raise ElementError('Could not recognize element number {0}.'.format(atom_num))
        atoms[symbol] = atoms.get(symbol, 0) + 1
    return atoms

In [ ]:
def get_rmg_conformer_from_logs(label,
                                 energy_log_path,
                                 freq_log_path,
                                 level_of_theory,
                                 freq_scale=1,
                                 multiplicity=1,
                                 molecule=None,
                                 use_atom_corrections=True,
                                 use_bond_corrections=False,):
    
    energy_log = ess_factory(energy_log_path)
    freq_log = ess_factory(freq_log_path)
    
    # Get the coords, atom_numbers, and mass
    coords, number, mass = freq_log.load_geometry()
    # Get the symmetry info of the molecule
    external_symmetry, optical_isomers = get_symmetry(coords, number)
    
    # This script will automatically assign modes but with wrong E0 and unscaled freqs
    conformer, unscaled_frequencies = freq_log.load_conformer(symmetry=external_symmetry,
                                                              spin_multiplicity=multiplicity,
                                                              optical_isomers=optical_isomers,
                                                              label=label)
    
    conformer.coordinates = (coords, "angstroms")
    conformer.number = number
    conformer.mass = (mass, "amu")
    
    zpe_scale_factor = freq_scale / 1.014
    e_electronic = energy_log.load_energy(zpe_scale_factor)

    # Atom energy corrections
    if use_atom_corrections:
        # Get atoms count
        atoms = get_element_counts(number)
        atom_corrections = get_atom_correction(level_of_theory, atoms)
    else:
        atom_corrections = 0

    # Bond energy corrections
    if use_bond_corrections:
        # Get bonds count
        try:
            bonds = molecule.enumerate_bonds()
            bond_corrections = get_bac(level_of_theory, bonds, coords, number,
                                       bac_type='p', multiplicity=multiplicity)
        except AttributeError:
            raise ValueError('Cannot get BAC, since argument ``molecule`` is not provided.')
    else:
        bond_corrections = 0

    e_electronic_with_corrections = e_electronic + atom_corrections + bond_corrections
    zpe = freq_log.load_zero_point_energy() * zpe_scale_factor if len(number) > 1 else 0
    conformer.E0 = ((e_electronic_with_corrections + zpe) * 0.001, 'kJ/mol')

    # Correct the frequencies
    for mode in conformer.modes:
        if isinstance(mode, HarmonicOscillator):
            mode.frequencies = (np.array(unscaled_frequencies) * freq_scale, "cm^-1")
    
    return conformer

In [ ]:
def get_rmg_conformer(label,
                      level_of_theory,
                      e_electronic,
                      frequencies,
                      coords,
                      numbers,
                      mass,
                      multiplicity=1,
                      freq_scale=1,
                      molecule=None,
                      use_atom_corrections=True,
                      use_bond_corrections=False,):
    
    external_symmetry, optical_isomers = get_symmetry(coords, numbers,)
    
    modes = []
    # Translational
    translation = IdealGasTranslation(mass=mass)
    modes.append(translation)
    
    # Rotational
    rotation = get_rotational_mode(coords, numbers,
                                   external_symmetry=external_symmetry)
    modes.append(rotation)
    
    # Vibrational
    frequencies = np.array(frequencies)
    frequencies = frequencies[frequencies >= 0]
    vibration = HarmonicOscillator(frequencies=(frequencies * freq_scale, "cm^-1"))
    modes.append(vibration)
    
    # Atom energy corrections
    if use_atom_corrections:
        atoms = get_element_counts(numbers)
        atom_corrections = get_atom_correction(level_of_theory, atoms)
    else:
        atom_corrections = 0

    # Bond energy corrections
    if use_bond_corrections:
        # Get bonds count
        try:
            bonds = molecule.enumerate_bonds()
            bond_corrections = get_bac(level_of_theory, bonds, coords, number,
                                       bac_type='p', multiplicity=multiplicity)
        except AttribureError:
            raise ValueError('Cannot get BAC, since argument ``molecule`` is not provided.')
    else:
        bond_corrections = 0
        
    e_electronic_with_corrections = e_electronic + atom_corrections + bond_corrections
    
    if len(numbers) > 1:
        zpe_scale_factor = freq_scale / 1.014
        scaled_zpe = 0.5 * constants.h * constants.c * constants.Na \
                     * np.sum(frequencies) * 100 * zpe_scale_factor
    else:
        scaled_zpe = 0

    e0 = ((e_electronic_with_corrections + scaled_zpe) * 0.001, 'kJ/mol')
    
    return Conformer(E0=e0, modes=modes,
                     spin_multiplicity=multiplicity,
                     optical_isomers=optical_isomers)

## 1. Get conformers from yaml file

In [ ]:
path = '/Users/xiaorui/Downloads/final_project_info.yml'

results = read_yaml_file(path)

In [ ]:
energy_level = results['level_of_theory']['fine_opt_freq']
freq_level = results['level_of_theory']['fine_opt_freq']

level_of_theory, freq_scale = get_lot_and_freq_scale(energy_level=energy_level,
                                                  freq_level=freq_level)

multiplicity = results['species']['multiplicity']

# Need to perceive molecule if using BAC
# if results['species']['is_ts']:
#     molecule = perceive_molecule
# else:
#     molecule=None

rmg_conformers = []
for hash_id in results['valid_conformer_hash_ids']:
    cur_conformer = results['conformers'][hash_id]
    xyz = cur_conformer['arc_xyz_after_fine_opt']
    coords = np.array(xyz['coords'])
    atom_numbers = [get_element(symbol).number for symbol in xyz['symbols']]
    mass = (sum([get_element(symbol).mass for symbol in xyz['symbols']])/constants.Na, 'kg')
    
    e_electronic = cur_conformer['energy']['end_of_fine_opt'] * 2625500  # hartree to J/mol 
    
    rmg_conformer = get_rmg_conformer(label=hash_id,
                                      level_of_theory=level_of_theory,
                                      e_electronic=e_electronic,
                                      frequencies=cur_conformer['frequencies'],
                                      coords=coords,
                                      numbers=atom_numbers,
                                      mass=mass,
                                      multiplicity=multiplicity,
                                      freq_scale=freq_scale,
                                      molecule=None,
                                      use_atom_corrections=True,
                                      use_bond_corrections=False)
    rmg_conformers.append(rmg_conformer)
    

Another option is use log file to obtain conformers

In [ ]:
path = '/Users/xiaorui/Downloads/log/opt_log/3_0x13b007fb2def9cdfa40ace809e4284e0_geom_fine_opt_freq.log'
conf = get_rmg_conformer_from_logs('',
                            energy_log_path=path,
                            freq_log_path=path,
                            level_of_theory=level_of_theory,
                            freq_scale=0.99*1.014,
                            multiplicity=2,
                            molecule=None,
                             use_atom_corrections=True,
                             use_bond_corrections=False,)

In [ ]:
conf

## 2. Use rmg conformers to compute MS properties
This part needs more careful checks to make sure the math is correct

### MS partition function

In [ ]:
e0_min = min([conf.E0.value_si for conf in rmg_conformers])
low_idx = np.argmin([conf.E0.value_si for conf in rmg_conformers])
low_conf = rmg_conformers[low_idx]

Uis = np.array([conf.E0.value_si - e0_min for conf in rmg_conformers])

Ts = np.arange(300., 2001., 10.)
Qs = np.zeros_like(Ts)
Qs_low = np.zeros_like(Ts)
for idx in range(Ts.shape[0]):
    T = Ts[idx]
    qis = np.array([conf.get_partition_function(T) for conf in rmg_conformers])
    Qs[idx] = np.sum(qis * np.exp(- Uis / constants.R / T))
    Qs_low[idx] = low_conf.get_partition_function(T)
    
plt.plot(Ts, Qs, label='MS')
plt.plot(Ts, Qs_low, label='Lowest')
plt.xlabel('T [K]')
plt.ylabel('Partition func')
plt.yscale('log')
plt.legend()

### MS enthalpy

In [ ]:
Hs = np.zeros_like(Ts)
Hs_low = np.zeros_like(Ts)
for idx in range(Ts.shape[0]):
    T = Ts[idx]
    His = np.array([conf.get_enthalpy(T) for conf in rmg_conformers])
    qis = np.array([conf.get_partition_function(T) for conf in rmg_conformers])
                   
    Hs[idx] = np.sum(qis * np.exp(- Uis / constants.R / T) * (His + Uis)) / Qs[idx]
    Hs_low[idx] = low_conf.get_enthalpy(T)
    
plt.plot(Ts, Hs/1000, label='MS')
plt.plot(Ts, Hs_low/1000, label='Lowest')
plt.xlabel('T [K]')
plt.ylabel('Enthalpy [kJ/mol]')
plt.legend()

### MS entropy

In [ ]:
Ss = np.zeros_like(Ts)
Ss_low = np.zeros_like(Ts)
for idx in range(Ts.shape[0]):
    T = Ts[idx]
    Ss[idx] = Hs[idx] / T - constants.R + constants.R * np.log(Qs[idx]) # Assume ideal gas
    Ss_low[idx] = low_conf.get_entropy(T)
    
plt.plot(Ts, Ss, label='MS',)
plt.plot(Ts, Ss_low, label='Lowest')
plt.xlabel('T [K]')
plt.ylabel('Enthalpy [J/mol]')
plt.legend()

### MS Gibbs free energy

In [ ]:
Gs = np.zeros_like(Ts)
Gs_low = np.zeros_like(Ts)
for idx in range(Ts.shape[0]):
    T = Ts[idx]
    Gs[idx] = Hs[idx] - T * Ss[idx]
    Gs_low[idx] = low_conf.get_free_energy(T)
    
plt.plot(Ts, Gs / 1000, label='MS',)
plt.plot(Ts, Gs_low / 1000, label='Lowest')
plt.xlabel('T [K]')
plt.ylabel('Gibbs [kJ/mol]')
plt.legend()

### MS heat capacity
I am not sure whether the current formula is correct.

In [ ]:
Cvs = np.zeros_like(Ts)
Cvs_low = np.zeros_like(Ts)
for idx in range(Ts.shape[0]):
    T = Ts[idx]
    His = np.array([conf.get_enthalpy(T) for conf in rmg_conformers])
    qis = np.array([conf.get_partition_function(T) for conf in rmg_conformers])
    Cvis = np.array([conf.get_heat_capacity(T) for conf in rmg_conformers])
    
    Cvs[idx] = np.sum(qis * np.exp(- Uis / constants.R / T) * Cvis) / Qs[idx]
    Cvs[idx] += 1 / constants.R / T**2 / Qs[idx] \
                * np.sum(qis * np.exp(- Uis / constants.R / T) * \
                        (His **2 + 2 * Uis * His + Uis ** 2))
    Cvs[idx] -= 1 / constants.R / T**2 * Hs[idx] ** 2
    Cvs_low[idx] = low_conf.get_heat_capacity(T)
    
plt.plot(Ts, Cvs, label='MS',)
plt.plot(Ts, Cvs_low, label='Lowest')
plt.xlabel('T [K]')
plt.ylabel('Heat Capacity [J/mol]')
# plt.yscale('log')
plt.legend()

## 3. MS Rate 

In [ ]:
def get_rmg_conformer_from_yml(path):
    # No bond correction at this moment
    
    results = read_yaml_file(path)
    energy_level = results['level_of_theory']['fine_opt_freq']
    freq_level = results['level_of_theory']['fine_opt_freq']

    level_of_theory, freq_scale = get_lot_and_freq_scale(energy_level=energy_level,
                                                         freq_level=freq_level)

    multiplicity = results['species']['multiplicity']

    rmg_conformers = []
    for hash_id in results['valid_conformer_hash_ids']:
        cur_conformer = results['conformers'][hash_id]
        xyz = cur_conformer['arc_xyz_after_fine_opt']
        coords = np.array(xyz['coords'])
        atom_numbers = [get_element(symbol).number for symbol in xyz['symbols']]
        mass = (sum([get_element(symbol).mass for symbol in xyz['symbols']])/constants.Na, 'kg')

        e_electronic = cur_conformer['energy']['end_of_fine_opt'] * 2625500  # hartree to J/mol 

        rmg_conformer = get_rmg_conformer(label=hash_id,
                                          level_of_theory=level_of_theory,
                                          e_electronic=e_electronic,
                                          frequencies=cur_conformer['frequencies'],
                                          coords=coords,
                                          numbers=atom_numbers,
                                          mass=mass,
                                          multiplicity=multiplicity,
                                          freq_scale=freq_scale,
                                          molecule=None,
                                          use_atom_corrections=True,
                                          use_bond_corrections=False)
        rmg_conformers.append(rmg_conformer)
    return rmg_conformers

def get_E0min_and_Q(Ts, conformers):
    E0_min = min([conf.E0.value_si for conf in conformers])
    Uis = np.array([conf.E0.value_si - E0_min for conf in conformers])
    Q = np.zeros_like(Ts)
    for idx in range(Ts.shape[0]):
        T = Ts[idx]
        qis = np.array([conf.get_partition_function(T) for conf in reactant])
        Q[idx] = np.sum(qis * np.exp(- Uis / constants.R / T))
    return E0_min, Q

In [ ]:
reactants_path = ['/Users/xiaorui/Downloads/final_project_info.yml',]
products_path = ['/Users/xiaorui/Downloads/final_project_info.yml',]
ts_path = '/Users/xiaorui/Downloads/final_project_info.yml'

In [ ]:
Temps = np.arange(300., 2001., 10.)

reactants = [get_rmg_conformer_from_yml(path) for path in reactants_path]
products = [get_rmg_conformer_from_yml(path) for path in products_path]
ts = get_rmg_conformer_from_yml(ts_path)

E0_reac = 0
Q_reac = 1
for reactant in reactants:
    E0_min, Q = get_E0min_and_Q(Temps, reactant)
    E0_reac += E0_min
    Q_reac *= Q
    
E0_prod = 0
for product in products:
    E0_min, _ = get_E0min_and_Q(Temps, product)
    E0_prod += E0_min

E0_TS, Q_TS = get_E0min_and_Q(Temps, ts)
low_idx = np.argmin([conf.E0.value_si for conf in ts])
low_ts_conf = list(read_yaml_file(ts_path)['conformers'].values())[low_idx]
neg_frequency = (low_ts_conf['negative_frequencies'][0], 'cm^-1')

eckart = Eckart(frequency = neg_frequency,
                E0_reac=(E0_reac, 'J/mol'),
                E0_TS=(E0_TS, 'J/mol'),
                E0_prod=(E0_prod, 'J/mol'))

dE0 = E0_TS - E0_reac

ks_w_tunnel = np.zeros_like(Temps)
ks_wo_tunnel = np.zeros_like(Temps)

for idx in range(Ts.shape[0]):
    ks_wo_tunnel[idx] = (constants.kB * T / constants.h * Q_TS[idx] / Q_reac[idx]) * np.exp(-dE0 / constants.R / T)
    ks_w_tunnel[idx] = ks_wo_tunnel[idx] * eckart.calculate_tunneling_factor(Ts[idx])